<a href="https://colab.research.google.com/github/Adi1116/Mindcase_AdityaSharma_IITKGP/blob/main/MindCase_AdityaSharma_Assignmnent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q pypdf
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers
!pip install -q llama-index

In [19]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [20]:
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-llama-cpp

In [21]:
!pip install cuda

ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [22]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
# Initializing LlamaCPP with a pretrained model.
llm = LlamaCPP(
    # I provide the option to specify the URL of a GGML model for automatic download.
    # The model_url parameter can be uncommented and filled accordingly.
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',

    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,

    # I limit the context window to allow for some flexibility.
    context_window=4096,

    # Additional arguments to be passed to the __call__() method.
    generate_kwargs={},

    # Additional arguments to be passed to the __init__() method.
    # Setting n_gpu_layers to -1 enables GPU usage.
    model_kwargs={"n_gpu_layers": -1},

    # Transforming inputs into Llama2 format.
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - k

In [23]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

# Loading data from the specified input file.
documents = SimpleDirectoryReader(
    input_files = ["/content/Mindcase Data.pdf"]
).load_data()

# Combining text from all loaded documents into a single Document object.
documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [24]:
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, load_index_from_storage
from llama_index.core import StorageContext
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):
  # Configuring the node parser for extracting sentence windows.
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  # Setting up the service context with necessary components.
  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # If the save directory doesn't exist, create and load the index.
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # If the save directory exists, load the existing index.
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [25]:
# Obtain the vector index.
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")

<ipython-input-24-c7eb83b9671c>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


In [26]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  # Configuring postprocessors for metadata replacement and sentence reranking.
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  # Creating a query engine from the sentence index with specified parameters.
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [27]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

**Example of queries that should be answered:**

- Explain the theme of the movie?
- Who are the characters?
- How many male and female characters are in the movie?
- Does the script pass the Bechdel test?
- What is the role of Deckard in the movie?

In [28]:
import sys

# Define a function to read input with timeout
def input_with_timeout(prompt, timeout):
    print(prompt, end='', flush=True)
    try:
        response = input()
        return response
    except KeyboardInterrupt:
        sys.exit(0)

# Define a flag to indicate whether the loop should continue running.
running = True

while running:
    try:
        # Input a query from the user.
        query = input("Enter your query (Press 'Esc' to exit): ")

        # Check if the input is 'Esc' to exit.
        if query.lower() == 'esc':
            print("Exiting the loop...")
            running = False
            continue

        # Query the engine with the input query.
        response = query_engine.query(query)

        # Print the response.
        print(response)
        print("\n")

    except KeyboardInterrupt:
        # If the user presses Ctrl+C, exit the loop.
        print("\nExiting the loop...")
        running = False

Enter your query (Press 'Esc' to exit): Explain the theme of the movie?



llama_print_timings:        load time =     393.82 ms
llama_print_timings:      sample time =      88.92 ms /   157 runs   (    0.57 ms per token,  1765.65 tokens per second)
llama_print_timings: prompt eval time =     393.21 ms /   372 tokens (    1.06 ms per token,   946.07 tokens per second)
llama_print_timings:        eval time =    3794.89 ms /   156 runs   (   24.33 ms per token,    41.11 tokens per second)
llama_print_timings:       total time =    4711.27 ms /   528 tokens


 Based on the provided context, it appears that the movie explores themes of identity, deception, and morality. The characters, Deckard (K) and Luv, are involved in a mission to locate and eliminate a replicant, but as they uncover more information, Deckard begins to question his own identity and past actions. Freysa's words suggest that there is a puzzle or mystery to be solved, and the recognition between Ana and K indicates a connection between their past experiences. The use of the term "replicant" implies the exploration of artificial intelligence and what it means to be human. Overall, the movie seems to delve into the complexities of identity, morality, and the consequences of one's actions.



Exiting the loop...


# New section

# Techniques Used

In the work above, I utilized several techniques and libraries to perform various tasks efficiently. Here's an overview of the key techniques used:

1. **LLAMA Index Integration:** The code integrates with the LLAMA Index library to handle document indexing and querying tasks. LLAMA Index provides efficient methods for processing and querying large collections of text data.

2. **PyTorch:** The PyTorch library is employed for handling tensor computations and machine learning tasks. It is used in conjunction with LLAMA Index for certain operations such as embedding generation and similarity calculations.

3. **Sentence Embeddings:** The code leverages pre-trained sentence embeddings to represent textual data in a high-dimensional vector space. These embeddings capture semantic information from text, enabling similarity comparisons and retrieval of relevant documents.

4. **Query Engine:** A query engine is constructed to facilitate user queries on the indexed documents. The engine utilizes similarity calculations and post-processing techniques to retrieve the most relevant results based on the user's input.

5. **Metadata Processing:** Metadata replacement and sentence reranking are applied as post-processing steps to enhance the quality of search results. These techniques involve modifying metadata associated with document nodes and reordering search results based on additional criteria.

By incorporating these techniques, the code demonstrates effective methods for document indexing, retrieval, and semantic search, making it suitable for various information retrieval tasks.


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
!streamlit hello




  Welcome to Streamlit. Check out our demo in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.224.66:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


  Stopping...
  Stopping...


In [ ]:
import streamlit as st

st.title("Blade Runner 2049 Expert")
user_input = st.text_input("Enter your questions about Blade Runner 2049 :", "")
if st.button("Submit"):
    try:
        response = query_engine(user_input)
        st.write(response)
    except Exception as e:
        st.write(f"An error occurred: {e}")

2024-04-18 17:17:05.301 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
! streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.224.66:8501

  Stopping...
  Stopping...
Exception ignored in: <module 'threading' from '/usr/lib/python3.10/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1518, in _shutdown
    def _shutdown():
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/bootstrap.py", line 55, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/server.py", line 405, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 315, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 798, in call_soon_threadsafe
    self._check_closed()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 515, in _check_closed
    raise RuntimeError('Ev